<a href="https://colab.research.google.com/github/amlagattuta/AI_Federated_Learning/blob/main/AIFinalPres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#We don't have client devices to use.
#Therefore, this is simulated federated learning.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.1.2 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.
flax 0.6.8 requires jax>=0.4.2, but you have jax 0.3.15 which is incompatible.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.15 which is incompatible.
arviz 0.15.1 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.


In [ ]:
!pip --quiet install tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.8/39.8 MB 19.5 MB/s eta 

In [ ]:
pip install tensorflow-federated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd

In [ ]:
# Load simulation data. Only using train data
source, _ = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
def client_data(n):
  return tf.data.Dataset.from_tensor_slices(
      #Source[1] represents the label an image has
      #So this line is subsetting all the images that belong to label N
      #Therefore, client_data[0], the first group, are all images that belong to category 0 (eg. dress)
      #This makes sense, for example if you have clients that specialize in a form of apparel. One client may just sell dresses while another just sells coats.
      (tf.reshape(source[0][source[1] == n], [-1, 784]),
       source[1][source[1] == n])
  ).repeat(10).batch(20)
  #Those images are flattened with .reshape()
  #These tuples of (image vector, label) are repeated 10 times to be used for training
  #This  dataset is put into batches with 20 elements each

In [ ]:
#Pick a subset of client devices to participate in training.
#Here we are picking three categories of apparel
#They are three different groups of data, but each group is just one category
train_data = [client_data(n) for n in range(3)]

In [ ]:
# Wrap a Keras model for use with TFF.
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, tf.nn.softmax, input_shape=(784,),
                            kernel_initializer='zeros')
  ])
  return tff.learning.models.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Simulate a few rounds of training with the selected client devices.
trainer = tff.learning.algorithms.build_weighted_fed_avg(
  model_fn,
  client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.01))
state = trainer.initialize()
for _ in range(7):
  result = trainer.next(state, train_data)
  state = result.state
  metrics = result.metrics
  print(metrics['client_work']['train']['loss'])
  #First loss metric here is so good because the model is only seeing one category
  #Loss metric gradually increases as new categories are introduced because there are more options for the model to use
  #After 3 rounds, the loss function improves dramatically because the model continues to train but no longer receives new categories to learn

0.0007684818
0.17754644
0.23579596
0.496357
0.16631602
0.13366593
0.2512209


**All code in one cell**

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
# Load simulation data.
source, _ = tf.keras.datasets.fashion_mnist.load_data()
def client_data(n):
  return tf.data.Dataset.from_tensor_slices(
      (tf.reshape(source[0][source[1] == n], [-1, 784]),
       source[1][source[1] == n])
  ).repeat(10).batch(20)
# Pick a subset of client devices to participate in training.
#this determines number of categories from fashion mnist dataset we use
train_data = [client_data(n) for n in range(10)]
# Wrap a Keras model for use with TFF.
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, tf.nn.softmax, input_shape=(784,),
                            kernel_initializer='zeros')
  ])
  return tff.learning.models.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Simulate a few rounds of training with the selected client devices.
trainer = tff.learning.algorithms.build_weighted_fed_avg(
  model_fn,
  client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.01))
state = trainer.initialize()
#n here determines how many simulations ie how many loss values are outputted. If this number is less than the number of client devices then we are missing out on output data
#if you simulate past the number of client devices, the loss value should improve because it is now only training on categories it has previously seen, not getting any new information
for _ in range(15):
  result = trainer.next(state, train_data)
  state = result.state
  metrics = result.metrics
  print(metrics['client_work']['train']['loss'])

0.00076848187
3.2614002
4.8429456
4.8643184
4.8412385
3.431396
3.2360978
3.232224
3.2577765
1.7308906
1.6257985
3.304713
3.3569214
4.8727727
3.2426803
